In [7]:
import os
import sys
%pip install -q ipywidgets
import ipywidgets as widgets

directorio = '../Results'
# para colab
# directorio = '/content' # si es un directorio en el que se subieron los archivos a colab o en drive agregar ruta

# Leer archivos xlsx dentro de la carpeta Data
files = os.listdir(directorio)
files = [f for f in files if f.endswith('.xlsx')]

# Seleccionar archivo a leer
dropdown = widgets.Dropdown(
  options=[('Seleccionar archivo', None)] + [(f, f) for f in files],
  description='Archivo:',
  disabled=False,
)
nombre_archivo = '';
def on_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
    if change['new'] is not None:
      global nombre_archivo
      nombre_archivo = change['new']
      print('Archivo seleccionado:', nombre_archivo)

dropdown.observe(on_change)
display(dropdown)


Note: you may need to restart the kernel to use updated packages.


Dropdown(description='Archivo:', options=(('Seleccionar archivo', None), ('04_clusters_sin_outliers.xlsx', '04…

Archivo seleccionado: 04_clusters_sin_outliers.xlsx


In [10]:
import pandas as pd

# Leer archivo seleccionado
path = os.path.join(directorio, nombre_archivo)

df = pd.read_excel(path)

# get numerical features in the DataFrame
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns
# quitar columnas que no son features
numerical_features = numerical_features.drop(['cluster_kmeans', 'cluster_hc'])

In [11]:
# from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols

# for feature in numerical_features:
#   df_cluster1 = df[df['cluster_kmeans'] == 1][feature]
#   df_cluster2 = df[df['cluster_kmeans'] == 2][feature]
#   df_cluster3 = df[df['cluster_kmeans'] == 3][feature]

#   # Realizar ANOVA
#   statistic, pvalue = f_oneway(df_cluster1, df_cluster2, df_cluster3)
#   print('Feature:', feature)
#   print('Estadístico F:', statistic)
#   print('Valor p:', pvalue)
#   print('')

## Realizar ANOVA con statsmodels para cada feature
for feature in numerical_features:
  model = ols(f'Q("{feature}") ~ C(cluster_kmeans)', data=df).fit()
  aov_table = sm.stats.anova_lm(model, typ=2)
  print('Feature:', feature)
  print(aov_table)
  print('')

Feature: alumnos
                         sum_sq      df           F        PR(>F)
C(cluster_kmeans)  31318.642100     2.0  192.041362  1.092506e-71
Residual           84639.970772  1038.0         NaN           NaN

Feature: 1p_total_actividades_asignadas
                         sum_sq      df           F         PR(>F)
C(cluster_kmeans)   5797.927481     2.0  300.311136  1.235667e-103
Residual           10020.022567  1038.0         NaN            NaN

Feature: 1p_total_actividades_evaluativas
                        sum_sq      df          F         PR(>F)
C(cluster_kmeans)  5316.728947     2.0  334.68854  6.721146e-113
Residual           8244.627441  1038.0        NaN            NaN

Feature: 1p_porcentaje_evidencias_calificadas
                      sum_sq      df           F         PR(>F)
C(cluster_kmeans)  44.570775     2.0  614.618637  8.007510e-177
Residual           37.636725  1038.0         NaN            NaN

Feature: 1p_total_actividades_calificadas
                       

In [12]:
# comparacion de medias
from statsmodels.stats.multicomp import pairwise_tukeyhsd

for feature in numerical_features:
  tukey = pairwise_tukeyhsd(endog=df[feature], groups=df['cluster_kmeans'], alpha=0.05)
  print('Feature:', feature)
  print(tukey.summary())
  print('')

# from scipy.stats import kruskal

# for feature in numerical_features:
#   df_cluster1 = df[df['cluster_kmeans'] == 1][feature]
#   df_cluster2 = df[df['cluster_kmeans'] == 2][feature]
#   df_cluster3 = df[df['cluster_kmeans'] == 3][feature]

#   # Realizar Kruskal-Wallis
#   statistic, pvalue = kruskal(df_cluster1, df_cluster2, df_cluster3)
#   print('Feature:', feature)
#   print('Estadístico H:', statistic)
#   print('Valor p:', pvalue)
#   print('')


Feature: alumnos
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper   reject
-----------------------------------------------------
     1      2  -12.863   0.0 -14.5795 -11.1464   True
     1      3  -3.2778 0.002  -5.5398  -1.0157   True
     2      3   9.5852   0.0   7.7198  11.4507   True
-----------------------------------------------------

Feature: 1p_total_actividades_asignadas
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
     1      2  -4.3243   0.0  -4.915 -3.7337   True
     1      3  -8.0407   0.0  -8.819 -7.2624   True
     2      3  -3.7164   0.0 -4.3582 -3.0745   True
---------------------------------------------------

Feature: 1p_total_actividades_evaluativas
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
 